In [1]:
import pickle
from pathlib import Path
import pandas as pd 
import os
import sys
import numpy as np
import getpass
import matplotlib.pyplot as plt

In [2]:
#add the github path that will run the notebooks: 
username = getpass.getuser() # gets the username from the current comp 
print (username)

gilmandelbaum


In [3]:
sys.path.insert(0,'/Users/{:}/github/analysis-pipeline-for-photometry_ex/Nb_data_set/3.data_set_correlations'.format(username))

In [4]:
sys.path[0]

'/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/3.data_set_correlations'

In [5]:
#import data_set_correlations_helper_functions as dschf

# Change based on analysis: 

In [6]:
seq_str = '0a1b2a3b4abcd5a6a7a'
period_of_interest = "combined_periods_1"

In [7]:
HowManyBack=1
folderName="right_dual_color_photo_full_data_set_after_inspection" #folder where the plot data_set will be saved 
path = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/2.dual_color_photo_full_data_set_analysis_3/"

# Folder name of analysis: 

In [8]:
analysis_type ="data_set_correlations"

In [9]:
sys.executable

'/anaconda3/bin/python3'

# Import the data and where to save it 

In [10]:
path_to_save = path+"/"+folderName+"/"+seq_str #also for loading

In [11]:
path_to_save

'/Volumes/GilInVivoPaper.large/2.Photometry_in_str/2.dual_color_photo_full_data_set_analysis_3//right_dual_color_photo_full_data_set_after_inspection/0a1b2a3b4abcd5a6a7a'

In [12]:
# stash result in a cache directory, for how many back. 
cache_dir_plt = os.path.join(path_to_save,analysis_type,period_of_interest)

In [13]:
# stash result in a cache directory, for how many back. 
cache_dir_plt = os.path.join(path_to_save,analysis_type,period_of_interest)
if not os.path.exists(cache_dir_plt):
    os.makedirs(cache_dir_plt)

# Import data_trial

In [14]:
%%time
root = Path(path_to_save)
d = "data_object_data_trial"+"_"+seq_str+'.pickle'
my_path = root / d 
fileToOpen = open(my_path, 'rb')
data_trial_sessions = pickle.load(fileToOpen)

CPU times: user 54.5 ms, sys: 12.3 ms, total: 66.7 ms
Wall time: 180 ms


# Extract selection time 

In [15]:
def extract_behaviorFeature_perSession(data_trial_sessions,behavior_measure):
    behavior_measure_list=[]
    for data_trial in data_trial_sessions:
        behavior_measure_list.append(pd.DataFrame(data_trial[behavior_measure]))
    return behavior_measure_list

In [16]:
data_trial_sessions_rt=extract_behaviorFeature_perSession(data_trial_sessions,'tSelection')

# Import data_photometry

In [17]:
%%time
root = Path(path_to_save)
d = "data_object_photometry"+"_"+seq_str+'.pickle'
my_path = root / d 
fileToOpen = open(my_path, 'rb')
data_set = pickle.load(fileToOpen)

CPU times: user 16.5 s, sys: 1.94 s, total: 18.4 s
Wall time: 37 s


# generate data sets of interest

In [18]:
def extract_data_of_interest_ipsi_contra_per_session_helper (PhotoData_perTrial_channels,rl,tt,period,cell_type):
    ipsi_next = PhotoData_perTrial_channels[rl][0][tt][period][cell_type]
    contra_next = PhotoData_perTrial_channels[rl][1][tt][period][cell_type]
    return (ipsi_next,contra_next)
    


def extract_data_of_interest_ipsi_contra_data_set (PhotoData_perTrial_channels_data_set,
                                                   rl,tt,period,cell_type): 
    
    ipsi_contra_data_set = []
    
    for PhotoData_perTrial_channels in PhotoData_perTrial_channels_data_set:
        ipsi_contra = extract_data_of_interest_ipsi_contra_per_session_helper (PhotoData_perTrial_channels,rl,tt,period,cell_type)
        ipsi_contra_data_set.append(ipsi_contra)
        
    return (ipsi_contra_data_set)

In [19]:
r_ls_period_d1_data_set= extract_data_of_interest_ipsi_contra_data_set(data_set,
                                                                           0,0,
                                                                           period_of_interest,"d1 R")

In [20]:
r_ls_period_d2_data_set= extract_data_of_interest_ipsi_contra_data_set(data_set,
                                                                           0,0,
                                                                           period_of_interest,"d2 R")

In [21]:
r_wr_period_d1_data_set= extract_data_of_interest_ipsi_contra_data_set(data_set,
                                                                           0,3,
                                                                           period_of_interest,"d1 R")

In [22]:
r_wr_period_d2_data_set= extract_data_of_interest_ipsi_contra_data_set(data_set,
                                                                           0,3,
                                                                           period_of_interest,"d2 R")

In [35]:
def extract_signal_per_session (data_state_sessions,function):
    maxSig_sessions=[]
    
    for session in data_state_sessions:
        max_ipsi_next = function(session[0])
        max_contra_next = function(session[1])
        
        maxSig_sessions.append((max_ipsi_next,max_contra_next))

    return maxSig_sessions

In [ ]:
# def extract_mean_max_signal(session_ipsi_or_contra):
    
#     list_max_value=[]
#     df = pd.DataFrame()
    
#     idx_max_signal_list = list(session_ipsi_or_contra.loc[200:500,:].idxmax())
    
#     for c,i in enumerate (idx_max_signal_list): #c is the count, i is the index of max value in each session avg 
#         max_signal_per_trial = session_ipsi_or_contra.iloc[(i-10):(i+11),c].mean()
#         list_max_value.append(max_signal_per_trial)
        
#     df['max_signal'] = list_max_value
    
#     trial_number = list(session_ipsi_or_contra.keys())
#     df.index = trial_number
    
#     return df

In [66]:
def area_under_curve_per_session(session_ipsi_or_contra):
    
    list_area_under_curve=[]
    df = pd.DataFrame()
    data_ipsi_contra_after_choice_lick = session_ipsi_or_contra.loc[200:500,:]
    
    
    for column in session_ipsi_or_contra:
        
        auc_trial=np.trapz(data_ipsi_contra_after_choice_lick[column])
        list_area_under_curve.append(auc_trial)
    
    df['area_under_curv'] = list_area_under_curve
    trial_number = list(session_ipsi_or_contra.keys())
    df.index = trial_number

    
    return df

In [67]:
r_ls_period_d1_maxSig=extract_signal_per_session(r_ls_period_d1_data_set,area_under_curve_per_session)

In [69]:
r_ls_period_d2_maxSig=extract_signal_per_session(r_ls_period_d2_data_set,area_under_curve_per_session)

In [70]:
r_wr_period_d1_maxSig=extract_signal_per_session(r_wr_period_d1_data_set,area_under_curve_per_session)

In [71]:
r_wr_period_d2_maxSig=extract_signal_per_session(r_wr_period_d2_data_set,area_under_curve_per_session)

In [72]:
def combine_resTime_maxSignal(resTime,maxSignal):
    
    trial_num=list(maxSignal.index) #get the index of the max signal (this is the trial numbers that we
    #are intersted in). 
    
    resTime_list = resTime.loc[trial_num] #get the response time of interest as a data frame 
    
    #combine the above:
    df_resTime_maxSig = pd.concat([resTime_list,maxSignal],axis=1)

    df_resTime_maxSig = df_resTime_maxSig
    
    return df_resTime_maxSig

In [73]:
def get_maxSig_resTime(processed_photo_sessions, processed_beh_sessions,column_header):
    df_time_max_signal_ipsi= pd.DataFrame()
    df_time_max_signal_contra = pd.DataFrame()   
    
    for photo,beh in zip(processed_photo_sessions,processed_beh_sessions):
        

        
        df_ipsi = combine_resTime_maxSignal(beh, photo[0])
        df_contra = combine_resTime_maxSignal(beh, photo[1])

        df_time_max_signal_ipsi = pd.concat([df_time_max_signal_ipsi, df_ipsi])
        df_time_max_signal_contra = pd.concat([df_time_max_signal_contra, df_contra])       
    
    
    
    df_time_max_signal_ipsi = df_time_max_signal_ipsi.reset_index().drop(["index"],axis=1)
    df_time_max_signal_contra = df_time_max_signal_contra.reset_index().drop(["index"],axis=1)
    
    
    #make second header: 
    df_time_max_signal_ipsi.columns = [[(column_header+"_ipsi"),(column_header+"_ipsi")], df_time_max_signal_ipsi.columns]
    df_time_max_signal_contra.columns = [[(column_header+"_contra"),(column_header+"_contra")], df_time_max_signal_contra.columns]


    
    
    return(df_time_max_signal_ipsi,df_time_max_signal_contra)        

In [74]:
r_ls_period_d1_resTime_maxSig=get_maxSig_resTime(r_ls_period_d1_maxSig, data_trial_sessions_rt,"r_ls_d1")

In [75]:
r_ls_period_d2_resTime_maxSig=get_maxSig_resTime(r_ls_period_d2_maxSig,data_trial_sessions_rt,"r_ls_d2")

In [76]:
r_wr_period_d1_resTime_maxSig=get_maxSig_resTime(r_wr_period_d1_maxSig,data_trial_sessions_rt,"r_wr_d1")

In [77]:
r_wr_period_d2_resTime_maxSig=get_maxSig_resTime(r_wr_period_d2_maxSig,data_trial_sessions_rt,"r_wr_d2")

In [78]:
result = pd.concat([r_ls_period_d1_resTime_maxSig[0],r_ls_period_d1_resTime_maxSig[1],
                    r_ls_period_d2_resTime_maxSig[0],r_ls_period_d2_resTime_maxSig[1],
                    r_wr_period_d1_resTime_maxSig[0],r_wr_period_d1_resTime_maxSig[1],
                    r_wr_period_d2_resTime_maxSig[0],r_wr_period_d2_resTime_maxSig[1]],axis=1)

In [79]:
result.head(5)

r_ls_d1_ipsi                 r_ls_d1_contra                 r_ls_d2_ipsi  \
    tSelection area_under_curv     tSelection area_under_curv   tSelection   
0         85.0       38.201940          415.0      127.914408         85.0   
1         90.0      216.301280          235.0      245.852677         90.0   
2         50.0      201.489504           65.0       79.455968         50.0   
3         50.0      254.966780          200.0      186.322464         50.0   
4        175.0       39.175651           75.0      -71.206372        175.0   

                  r_ls_d2_contra                 r_wr_d1_ipsi                  \
  area_under_curv     tSelection area_under_curv   tSelection area_under_curv   
0      113.261540          415.0     -201.615984          180      361.694835   
1      387.480851          235.0       56.716378           70      215.410659   
2      321.062216           65.0      194.554401          185      108.923088   
3      239.940068          200.0      -58.165379          230      300.902900   
4      172.193137           75.0       13.488871          185      259.613763   

  r_wr_d1_contra                 r_wr_d2_ipsi                 r_wr_d2_contra  \
      tSelection area_under_curv   tSelection area_under_curv     tSelection   
0           80.0      183.539990          180      325.840133           80.0   
1          215.0      237.014782           70      216.771767          215.0   
2           55.0       57.416272          185      132.885666           55.0   
3          210.0      161.788743          230      390.307669          210.0   
4           85.0      143.983054          185      410.716821           85.0   

                   
  area_under_curv  
0      429.199464  
1      613.551841  
2      424.864387  
3      395.304182  
4      503.158808

In [80]:
result.to_csv(cache_dir_plt+"/data_set_correlations_rt_vs_AUC_right_side.csv")